### Home Network Assistant - Creating an Agent
---

In this notebook we show you how to create your first sub-agent on Amazon Bedrock Agents.

Amazon Bedrock Agents enable generative AI applications to execute `multi-step` business tasks using natural language.

In our second example we will create a `Home Network` sub agent that will have access to an `API` spec and will be able to generate code to call the `API` based on the user request.

**Agent code generation and execution workflow**:

In this notebook, we will perform the following: Create an Agent, use the response generated from the knowledge base (from running notebook `0`) and generate code for the API spec. 

***The agent will have access to an action group with several tools. These tools will be accessible by the agent to get the user query, retrieve relevant data from the knowledge base using the query, identify the API spec needed to answer the user question and generate python code for it, save the code and execute the code to provide the user with the answer to their question.***

### Creating Agent

On this section we declare global variables that will be act as helpers during entire notebook and you will start to create your first agent.

In [ ]:
import os
import sys
import uuid
import json
import time
import boto3
import logging
from typing import Optional
from dotenv import load_dotenv

# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)
from globals import *

In [ ]:
# Load the environment variables that are defined in the ".env" file.
load_dotenv

In [ ]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# configure the sts client, the boto3 session and other variables
sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
account_id_suffix = account_id[:3]

### Importing helper functions

On following section, we're adding `bedrock_agent_helper.py` on Python path, so the files can be recognized and their functionalities can be invoked.

Those files contain helper classes totally focused on make labs experience smoothly.

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:

1. `create_agent`: Create a new agent and respective IAM roles
1. `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
1. `create_agent_alias`: Create an alias for this agent
1. `invoke`: Execute agent

In [ ]:
import sys
sys.path.insert(0, ".")
sys.path.insert(1, "..")
from utils.utils import *

region = get_aws_region()
logger.info(f"Detected AWS Region: {region}")
agent_suffix = f"{region}-{account_id_suffix}"
s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

# Import utility functions and helper functions for agents
from utils.utils import *
from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

### Load the config file
--- 

Load the config file that contains information on the models, data directories, etc.

In [ ]:
# Get the absolute path to the config file
BASE_DIR = os.path.abspath(sys.path[1])
CONFIG_FPATH = os.path.join(BASE_DIR, CONFIG_FNAME)
config_data = load_config(CONFIG_FPATH)
logger.info(f"Loaded config from local file system: {json.dumps(config_data, indent=2)}")

### Creating Agent
---

Create the `Home Network` assistant agent that will have access to API specs that it can refer to and generate code for the API of interest and execute the code.

In order to have accurate agents, it is important to set unambiguous instructions of what the agent should do and what it should not do. It is also important to provide clear definitions for when the agent should use the knowledge bases and action groups available to it.

This agent will have access to the output from the previously created KB and returns search results for the top `k` elements that are retrieved from the API spec.

In [ ]:
agent_description = """You are a Home Network assistant agent.
You help generate code for Home Network API operations based on user questions and knowledge base content."""

In [ ]:
instructions_path: str = os.path.join(config_data['dir_paths']['agent_instructions_prefix'], 
                                      config_data['dir_paths']['agent_instructions'].get('home_network_agent_instructions'))
agent_instruction = open(os.path.join(parent_dir, instructions_path), 'r').read()

In [ ]:
print(agent_instruction)

### Code Generation Prompt: Bedrock prompt management
---

Let's create our sample code generation prompt by leveraging on Prompt Management for Amazon Bedrock. This is the prompt that is used by the sub agent to generate the code when it calls the `generate_code` tool.

In [ ]:
def read_prompt_from_file(file_path: str) -> str:
    with open(file_path, 'r') as file:
        return file.read().strip()

prompt_file_path = os.path.join(
    config_data['dir_paths']['code_gen_prompts_prefix'],
    config_data['dir_paths']['code_gen_prompts'].get('home_network_code_generation_prompt')
)

absolute_prompt_fpath = os.path.join(
    parent_dir,
    prompt_file_path
)

prompt_template_code_gen: str = read_prompt_from_file(absolute_prompt_fpath)
print(f"Code generation prompt that will be saved in prompt management within Bedrock: {prompt_template_code_gen}")

In [ ]:
bedrock_agent = boto3.client(service_name = "bedrock-agent", region_name = region)
response = bedrock_agent.create_prompt(
    name = f"prompt-for-home-network-code-gen",
    description = "Code generation prompt template that is used by the home networking agent to generate code",
    variants = [
        {
            "name": "variantOne",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {
                            "name": "input"
                        },
                        {
                            "name": "output"
                        }
                    ],
                    "text": prompt_template_code_gen
                }
            },
            "templateType": "TEXT"
        }
    ],
    defaultVariant = "variantOne"
)

print(json.dumps(response, indent=2, default=str))
promptId = response["id"]
promptArn = response["arn"]
promptName = response["name"]
print(f"Prompt ID: {promptId}\nPrompt ARN: {promptArn}\nPrompt Name: {promptName}")

In [ ]:
# Now that we have a draft prompt, we can create a version from it.
response = bedrock_agent.create_prompt_version(
    promptIdentifier = promptId
)
print(json.dumps(response, indent=2, default=str))

### Creating Lambda

In order to enable the agent to execute tasks, we will create an AWS Lambda function that implements the tasks execution. We will then provide this lambda with tools that the agent will have access to while answering user questions.

In [ ]:
# create the function definitions
functions_def = [
    {
        "name": "query_knowledge_base",
        "description": "Queries the knowledge base with the user's query to fetch relevant API documentation and returns the relevant chunks",
        "parameters": {
            "query": {
                "description": "This is the user's query",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "generate_code",
        "description": "Generates Python code based on the knowledge base content and user query",
        "parameters": {
            "chunks": {
                "description": "List of relevant content chunks from the knowledge base",
                "required": True,
                "type": "array"
            },
            "query": {
                "description": "The original user query to provide context for code generation",
                "required": True,
                "type": "string"
            }, 
            "input_params": {
                "description": "JSON string containing input parameters needed to execute the generated code",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "save_generated_code",
        "description": "Saves the generated Python code to a temporary file",
        "parameters": {
            "code_content": {
                "description": "The generated Python code to be saved",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "execute_generated_code",
        "description": "Executes the saved Python code and returns the execution results",
        "parameters": {
            "file_path": {
                "description": "Path to the saved Python code file to execute",
                "required": True,
                "type": "string"
            }
        }
    }
]

In [ ]:
# Create agent
home_network_agent = agents.create_agent(
    HOME_NETWORK_AGENT_NAME,
    agent_description,
    agent_instruction,
    config_data['model_information']['doorbell_sub_agent_model'],
    kb_arns=[],
    code_interpretation=False
)

Now, we will use some lambda environment variables from the config file. These include the code generation model of choice, inference parameters, the prompt to generate the code and the code execution time out.

In [ ]:
# define the lambda environment variables
lambda_env_vars = config_data['code_generation_model_information']
lambda_env_vars['CODE_GEN_PROMPT_ID'] = promptId  
lambda_env_vars['HOME_NETWORK_AUTH_TOKEN'] = os.getenv("HOME_NETWORK_AUTH_TOKEN")

logger.info(f"Lambda environment variables: {json.dumps(lambda_env_vars, indent=2)}")


### Get any requirements to be installed in the custom container used within the lambda function
---

In this portion of the notebook, we will fetch the requirements to be installed within the custom container that will be used and attached to the lambda function. In this example, we are installing the `requests` library within the container set up.

The `requests` library will then be used to execute the generated code within the lambda so that it does not have to be installed every time before executing the generated code.

In [ ]:
# To configure more libraries that are required, mention them in the 'lambda_docker_set_up' within the 
# config.yaml file
lambda_function_libraries = config_data['lambda_docker_set_up']['libraries']
lambda_platform = config_data['lambda_docker_set_up']['platform']
logger.info(f"The libraries that will be installed within the custom container are as follows: {lambda_function_libraries}")

In [ ]:
agent_id = home_network_agent[0]
agents.wait_agent_status_update(agent_id)

agents.add_action_group_with_lambda(
    agent_name=HOME_NETWORK_AGENT_NAME,
    lambda_function_name=f"{HOME_NETWORK_AGENT_NAME}_lambda", 
    source_code_file=HOME_NETWORK_AGENT_LAMBDA_FUNCTION_NAME,
    agent_functions=functions_def,
    agent_action_group_name=HOME_NETWORK_ACTION_GROUP_NAME,
    agent_action_group_description="Functions to query KB, generate and execute code",
    lambda_function_libraries=lambda_function_libraries,
    platform=lambda_platform
)

In [ ]:
agents._bedrock_agent_client.prepare_agent(agentId=agent_id)
agents.wait_agent_status_update(agent_id)
agent_alias = agents._bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName="demo"
)

In [ ]:
%store -r
home_network_kb_id

In [ ]:
# Create a Lambda client and attach the API key as env variable to the lambda function
lambda_client = boto3.client('lambda')
lambda_function_name = f"{HOME_NETWORK_AGENT_NAME}_lambda"
environment_variables = {
    'HOME_NETWORK_KB_LAMBDA_FUNCTION_NAME': HOME_NETWORK_KB_LAMBDA_FUNCTION_NAME,
    'KB_ID': home_network_kb_id,
    'REGION': region, 
} | lambda_env_vars

response = lambda_client.update_function_configuration(
    FunctionName=lambda_function_name,
    Environment={
        'Variables': environment_variables
    },
)
logger.info(f"Updated the {lambda_function_name} with the required environment variables")

### Test the Home Networking agent
---

In this portion of the notebook, we will ask questions to the agent with parameters. First, we will call the lambda function and then with the output of the lambda function, the agent will be invoked.

In [ ]:
time.sleep(30)

## Question 1

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    """What is the signal strength of my porch camera?""",
    home_network_agent[0], enable_trace=True, session_id=session_id
)
print("====================")
print(response)

In [ ]:
%%time
response = agents.invoke(
    """The deviceId of the porch camera is madhurdummy2039.""",
    home_network_agent[0], enable_trace=True, session_id=session_id
)
print("====================")
print(response)

The auth token needs to be provided, so we see the following errors in the logs: 

```{'execution_result': {'stdout': "An error occurred: HTTPSConnectionPool(host='api.smarthomesecurity.example.com', port=443): Max retries exceeded with url: /v1/devices/cameras/madhurdummy2039/status (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fe6dbd6f620>: Failed to establish a new connection: [Errno -2] Name or service not known'))\n", 'stderr': '', 'return_code': 0, 'success': True}}```

In [ ]:
home_network_agent_alias_id, home_network_agent_alias_arn = agents.create_agent_alias(
    home_network_agent[0], 'v1'
)
home_network_agent_id = home_network_agent[0]

In [ ]:
home_network_agent_arn = agents.get_agent_arn_by_name(HOME_NETWORK_AGENT_NAME)
home_network_agent_id = home_network_agent[0]
home_network_kb = HOME_NETWORK_KB_NAME

%store home_network_agent_arn
%store home_network_agent_id
%store home_network_agent_alias_id
%store home_network_kb
%store home_network_agent_alias_arn